# Reactive air shock
In this report a code to compute the chemical relaxation past a shock is developed. [Aggiungere cose]

Firstly, as python is used, the required packages are imported and global variables are defined



In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
Kb  = 1.3806503e-23; # Boltzmann constant
amu = 1.66053904020e-27;

## Specie class definition

For each specie a common framework is defined which allows to compute thermodynamic derivatives, energy, temperature and other relevant values for the subsequent calculations starting from the data provided by Park and Zanardi.

Assuming that the different contributions can be computed indipendently, the temperature and CVs are defined as the sum of the translational-rotational, vibrational and electronic which are calculated as:

* Translational-rotational : 
    * If the molecule is monoatomic, it has no rotational dofs, then $ CV^t = \frac{3}{2} \times R_{gas} \times T $
    * If the molecule is not monoatomic, the rotational contribution must be added $ CV^{(rt)} = CV^t + R $
    
    Then, energy is computed as $e^{tr} = CV \times T$.
    
* Vibrational :
    The contribution of vibrational dofs to energy is computed by assuming armhonic oscillator potential. The energy can becomes: $e^{v} = R_{gas} \times \frac{\theta_v}{e^{\frac{\theta_v}{T}-1}} $ and the specific heat is computed by deriving with respect to T.
    
* Electronic :
    Given the partition function (Zanardi's tables), the energy due to electronic energy levels is computed as $ e = R_{gas}T^2 \frac{\partial}{\partial T} log(Z) = \frac{R_gas}{Z} \sum_i g_i \theta_i e^{-\frac{\theta_i}{T}}$. Analogously with the vibrational contribution the expression for the specific heat is obtained deriving the formula with respect to T.

In [3]:
class specie:
    def __init__(self, mmol, th_v, th_r, th_d):
        specie.th_v = th_v
        specie.th_r = th_r
        specie.th_d = th_d
        specie.electro_g = None
        specie.electro_th = None
        specie.e0 = None
        specie.e_rot = None
        specie.e_vib = None
        specie.e_ele = None
        specie.e = None
        specie.mmol = mmol
        specie.R = Kb/mmol/amu
        
    def energy(self, T):
        'Computes the energy and Cv as the sum of translational-rotational, vibrational and electronic contributions'
        e = 0
        Cv = 0
        
        # Traslational - rotational 
        if self.th_r > 0:
            Cv_rot = 3/2 * self.R + self.R
            
        else:
            Cv_rot = 3/2 * self.R
        e_rot  = Cv_rot * T 
        
        # Vibrational 
        if self.th_v > 0:
            CV_vib = self.R*((self.th_v/T)**2)*(np.exp(self.th_v/T))/(np.exp(self.th_v/T)-1)**2;
            e_vib  = self.R*self.th_v/(np.exp(self.th_v/T) - 1);
        else:
            CV_vib = 0
            e_vib  = 0
        
        # Electronic
        Z = np.sum(np.multiply(self.electro_g, np.exp(-self.electro_th/T)))
        Z1= np.sum(np.multiply(self.electro_g, np.exp(-self.electro_th/T), self.electro_th)) 
        Z2= np.sum(np.multiply(self.electro_g, np.exp(-self.electro_th/T), self.electro_th)) 
        e_ele  = self.R * Z1 / Z
        CV_ele = self.R/(T**2)*(-(e_ele/self.R)**2 + Z2 / Z)
        
        # Update Cv and e values
        CV = Cv_rot + CV_vib + CV_ele
        e  = e_rot  + e_vib  + e_ele
        
        return e, CV

Species are defined as objects of class specie and properties are defined according to the tables in Zanardi.

In [5]:
N2 = specie(28, 3395, 113200, 0)
N2.electro_g  = np.array([1, 3, 6, 6, 3, 1, 2, 2, 5, 1, 6, 6, 10, 6, 6])
N2.electro_th = np.array([0, 7.2231565e+04, 8.5778626e+04, 8.6050267e+04, 9.5351186e+04, 9.8056357e+04, 9.9682677e+04, 1.0489765e+05, 1.1164896e+05, 1.2258365e+05,
                 1.2488569e+05, 1.2824762e+05, 1.3380609e+05, 1.4042964e+05, 1.5049589e+05])


(372614.0593416891, 757.8742549736087)